In [1]:
using Revise
using RigidBodyDynamics
using DrakeVisualizer
using Plots
using RigidBodyTreeInspector

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/8c8a043c2f00369f3e7ae550277d2a54e59a34ad-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/8c8a043c2f00369f3e7ae550277d2a54e59a34ad-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

In [2]:
urdf = joinpath("..", "urdf", "panda", "panda_arm.urdf")
mechanism = parse_urdf(Float64, urdf)
state = MechanismState(mechanism)

MechanismState{Float64, Float64, Float64, …}(…)

In [20]:
function make_end_effector_controller(state::MechanismState,
                                      end_effector::RigidBody,
                                      end_effector_point::Point3D,
                                      desired_point::Point3D)
    
    mechanism = state.mechanism
    v̇ = similar(velocity(state))
    
    joint_path = path(mechanism, root_body(mechanism), end_effector) # from root to end effector
    Jp = point_jacobian(state, joint_path, transform(state, end_effector_point, root_frame(mechanism)))
        
    function controller!(τ, t, state)
        ef_point_in_world = transform_to_root(state, end_effector) * end_effector_point
        point_jacobian!(Jp, state, joint_path, ef_point_in_world)
        des_point_in_world = transform(state, desired_point, root_frame(mechanism))
        Kp = 10.
        Kd = 5.
        Δp = des_point_in_world - ef_point_in_world
        v̇ .= Kp * Array(Jp)' * Δp.v .- Kd .* velocity(state)
        τ .= inverse_dynamics(state, v̇)
    end
end

desired_point_v = [-.5, .6, .6]
end_effector = findbody(mechanism, "panda_link8")
end_effector_point = Point3D(default_frame(end_effector), [0.,.025,0.])
desired_point = Point3D(root_frame(mechanism), desired_point_v)
point_ctrl! = make_end_effector_controller(state, end_effector, end_effector_point, desired_point)

(::controller!) (generic function with 1 method)

In [21]:
set_configuration!(state, zeros(num_positions(mechanism)))
set_velocity!(state, zeros(num_velocities(mechanism)))
setdirty!(state)
final_time = 20.
ts, qs, vs = simulate(state, final_time, point_ctrl!; Δt = 1e-3);

In [8]:
qs

20001-element Array{RigidBodyDynamics.CustomCollections.SegmentedVector{RigidBodyDynamics.JointID,Float64,Base.OneTo{RigidBodyDynamics.JointID},Array{Float64,1}},1}:
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]                                                     
 [2.19927e-8, 5.45338e-8, 2.19927e-8, -1.58957e-8, 2.19927e-8, -1.53469e-8, -1.03106e-33]
 [8.79414e-8, 2.18063e-7, 8.79414e-8, -6.35616e-8, 8.79414e-8, -6.13671e-8, 3.80644e-23] 
 [1.97802e-7, 4.90477e-7, 1.97802e-7, -1.42966e-7, 1.97802e-7, -1.3803e-7, 9.5161e-23]   
 [3.51531e-7, 8.71669e-7, 3.51531e-7, -2.54077e-7, 3.51531e-7, -2.45305e-7, 1.7129e-22]  
 [5.49085e-7, 1.36153e-6, 5.49084e-7, -3.96864e-7, 5.49084e-7, -3.83161e-7, 2.09354e-22] 
 [7.9042e-7, 1.95995e-6, 7.90418e-7, -5.71293e-7, 7.90418e-7, -5.51568e-7, 2.56935e-22]  
 [1.07549e-6, 2.66682e-6, 1.07549e-6, -7.77335e-7, 1.07549e-6, -7.50494e-7, 3.42579e-22] 
 [1.40426e-6, 3.48204e-6, 1.40425e-6, -1.01496e-6, 1.40425e-6, -9.79911e-7, 4.47257e-22] 
 [1.77667e-6, 4.40549e-6

In [10]:
DrakeVisualizer.any_open_windows() || (DrakeVisualizer.new_window(); sleep(1));
geometries = visual_elements(mechanism, URDFVisuals(urdf))
vis = Visualizer(mechanism, geometries);

In [15]:
desired_point_vis = Point3D(root_frame(mechanism), desired_point_v)
addgeometry!(vis, mechanism, desired_point_vis; radius=0.025)

In [22]:
RigidBodyTreeInspector.animate(vis, mechanism, ts, qs; realtimerate = 1.);